# Radar-based flood mapping
*Following the procedure from [UN-SPIDER radar-based flood mapping](https://github.com/UN-SPIDER/radar-based-flood-mapping)*

## User Input

Please specify in the code cell below i) the polarisation to be processed, ii) whether data shall be downloaded from the <a href="https://scihub.copernicus.eu/">Copernicus Open Access Hub</a> with respective sensing period and login details, and iii) whether intermediate results should be plotted during the process.

In [ ]:
# polarisations to be processed
polarisations = 'VH'                              # 'VH', 'VV', 'both'

# download image from Copernicus Open Access Hub
dlinfo = {
    'period_start'      : [2022, 1, 15],           # format: [Year, Month, Day] e.g. DAY AFTER FLOOD HAPPENED 
    'period_stop'       : [2022, 1, 25],           # format: [Year, Month, Day] e.g. WEEK AFTER FLOOD HAPPENED
    'username'          : 'username',             # username for login
    'password'          : 'password'              # password for login
}

# show intermediate results if set to 'True'
showmaps = True                               # 'True', 'False'

## Map flood

1. Downloads satellite image closest to flood date
2. Processes image
3. Writes output

In [ ]:
from main import *
mapflood(polarisations, dlinfo, showmaps)